In [124]:
from copy import copy

In [624]:
VAR_INDEX = 0

# Abstract class Formula
class Formula(): 
    def __init__(self, args, negated=False):
        self.negated = negated
        self.args = args  
    def __neg__(self): 
        new_object = copy(self)
        new_object.negated = not new_object.negated
        return new_object 
    def __add__(self, other): return Disjunction(self, other)
    def __mul__(self, other): return Conjunction(self, other)
    def __rshift__(self, other): return Implication(self, other)
    def __mod__(self, other): return Equivalence(self, other)
    def __lt__(self, other):
        if type(self) == Var and type(other) == Var:
            return self.name < other.name
        if type(self) == Var and (type(other) == Conjunction or type(other) == Disjunction):
            return self.name <= other.args[0].name
        if  (type(self) == Conjunction or type(self) == Disjunction) and type(other) == Var:
            return self.args[0].name < other.name
        if  (type(self) == Conjunction and type(other) == Conjunction) or (type(self) == Disjunction and type(other) == Disjunction):
            self_names = []
            other_names = []
            for arg in self.args:
                self_names.append(arg.name)
            for arg in other.args:
                other_names.append(arg.name)
            print(self_names, other_names)
            return self_names < other_names
        
        
# Abstract class Connective 
class Connective(Formula): 
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Conjunction
class Conjunction(Connective):
    pass

# Disjunction
class Disjunction(Connective): 
    pass

# Implication
class Implication(Formula):
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Equivalence
class Equivalence(Formula): 
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Variable
class Var(Formula):
    def __init__(self, name, specific_negated=False):
        super().__init__(None, negated=specific_negated)        
        global VAR_INDEX
        self.index = VAR_INDEX
        VAR_INDEX += 1
        self.name = name

In [265]:
def remove_imp_eq(formula):
    if type(formula) == Var:
        return formula
    
    if type(formula) == Conjunction:
        expr = remove_imp_eq(formula.args[0]) * remove_imp_eq(formula.args[1])
    
    if type(formula) == Disjunction:
        expr = remove_imp_eq(formula.args[0]) + remove_imp_eq(formula.args[1])
    
    if type(formula) == Implication:
        expr = remove_imp_eq(-(formula.args[0])) + remove_imp_eq(formula.args[1])
    
    if type(formula) == Equivalence:
        expr = remove_imp_eq(formula.args[0]) * remove_imp_eq(formula.args[1]) + \
               remove_imp_eq(-(formula.args[0])) * remove_imp_eq(-(formula.args[1]))
    expr.negated = formula.negated
    return expr

In [266]:
f = (a % a) >> (b % b)
r = remove_imp_eq(f)
print(r.args[0].name, r.args[0].negated)
print(r.args[1].name, r.args[1].negated)
print(type(r))

AttributeError: 'Disjunction' object has no attribute 'name'

In [9]:
def de_morgan(formula):
    if type(formula) == Var:
        return formula
    if type(formula) == Disjunction :
        if formula.negated:
            return de_morgan(-formula.args[0]) * de_morgan(-formula.args[1])
        else:
            return de_morgan(formula.args[0]) + de_morgan(formula.args[1])
    if type(formula) == Conjunction:
        if formula.negated:
            return de_morgan(-formula.args[0]) + de_morgan(-formula.args[1])
        else:
            return de_morgan(formula.args[0]) * de_morgan(formula.args[1])
    
        
a = Var('a')
b = Var('b')
c = Var('c')
d = Var('d')

f = -(a * b) + --(c * d) + a + --(a + a)
d = de_morgan(f)

In [10]:
d.args[0].args[0].args[1]

In [219]:
def flatten_list(l):
    flat_list = []
    for list_elem in l:
        if type(list_elem) == list:
            for item in list_elem:
                flat_list.append(item)
        else:
            flat_list.append(list_elem)
    return flat_list

def remove_associativity(formula):
    if type(formula) != Var: 
        for i, arg in enumerate(formula.args):
            if (type(formula) == Disjunction and type(formula.args[i]) == Disjunction) or \
            (type(formula) == Conjunction and type(formula.args[i]) == Conjunction):
                formula.args[i] = formula.args[i].args
                formula.args = flatten_list(formula.args)
                remove_associativity(formula)  
        for i, arg in enumerate(formula.args):
            remove_associativity(arg)

In [241]:
form = a * (a + (b*b*b*b) + a) * a
remove_associativity(form)

form.args[1].args[1].args

In [259]:
form = (a+a)*(b*b)
remove_associativity(form)
form.args

In [571]:
def sort_leaves(formula):
    for i, arg in enumerate(formula.args):
        if type(arg) != Var:
            sort_leaves(arg)        
    for i, arg in enumerate(formula.args):
        if type(arg) != Var:
            return
    formula.args.sort()

In [564]:
a = Var('a')
b = Var('b')
c = Var('c')
d = Var('d')
e = Var('e')
a,b,c,d,e

(<__main__.Var at 0x1539922ae80>,
 <__main__.Var at 0x1539960ef60>)

In [572]:
f = d * (c + b) * e * (b + a) 
remove_associativity(f)
sort_leaves(f)
f.args[3].args[0].name

'a'